In [1]:
import os
import pandas as pd

In [2]:
%pwd

'/Users/harkiratchahal/Desktop/Coding/Tutorials/mlops_2/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/harkiratchahal/Desktop/Coding/Tutorials/mlops_2'

In [12]:
from dataclasses import dataclass
from pathlib import Path

In [13]:
@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [14]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [15]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [16]:
import os
from mlProject import logger
from sklearn.model_selection import train_test_split
from mlProject import logger
from sklearn.preprocessing import LabelEncoder
from typing import Tuple
import pandas as pd

In [42]:
class DataTransformation:
    def __init__(self, config):
        self.config = config

    def handle_null_values(self, data: pd.DataFrame) -> pd.DataFrame:
        """Handle null values in the data."""
        logger.info("Handling Null Values...")
        return data.fillna(0, inplace=True)
    
    def drop_columns(self, data: pd.DataFrame, columns: list) -> pd.DataFrame:
        """Drop specified columns."""
        logger.info(f"Dropping Columns: {', '.join(columns)}")
        return data.drop(columns, axis=1, inplace=True)

    def handle_outliers(self, data: pd.DataFrame, column: str) -> pd.DataFrame:
        """Handle outliers in a specified column."""
        logger.info(f"Handling Outliers in Column: {column}")
        Q1 = data[column].quantile(0.25)
        Q3 = data[column].quantile(0.75)
        IQR = Q3 - Q1
        filter = (data[column] >= Q1 - 1.5 * IQR) & (data[column] <= Q3 + 1.5 * IQR)
        return data.loc[filter]

    def encode_labels(self, data: pd.DataFrame, columns: list) -> pd.DataFrame:
        """Label encode specified columns."""
        logger.info(f"Encoding Labels for Columns: {', '.join(columns)}")
        label_encoder = LabelEncoder()
        for col in columns:
            data[col] = label_encoder.fit_transform(data[col])
        return data
    
    def dummy_encode(self, data: pd.DataFrame, columns: list) -> pd.DataFrame:
        """Dummy encode specified columns."""
        logger.info(f"Dummy Encoding Columns: {', '.join(columns)}")
        for col in columns:
            dummies = pd.get_dummies(data[col], prefix=f'dummy_{col}', drop_first=True)
            dummies = dummies.astype(int)
            data = pd.concat([data, dummies], axis=1)
            data = data.drop(columns=[col])
        return data

    
    def split_and_save(self, data: pd.DataFrame, test_size: float) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """Split data into train and test sets and save them as CSV files."""
        logger.info(f"Splitting Data with Test Size: {test_size}")
        train, test = train_test_split(data, test_size=test_size)
        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)
        logger.info("Data has been split and saved.")
        return train, test

    def perform_data_transformation(self):
        """Perform data transformations and split data into train and test."""
        logger.info("Starting Data Transformation Process...")
        data = pd.read_csv(self.config.data_path)
        self.handle_null_values(data)
        self.drop_columns(data, ['sl_no', 'ssc_b', 'hsc_b'])
        data = self.handle_outliers(data, 'hsc_p')
        data = self.encode_labels(data, ['gender', 'workex', 'specialisation', 'status'])
        data = self.dummy_encode(data, ['hsc_s', 'degree_t'])
        train, test = self.split_and_save(data, test_size=0.25)
        logger.info("Data Transformation Process Completed.")

In [43]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.perform_data_transformation()
except Exception as e:
    raise e


[2023-10-03 01:07:39,246: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-10-03 01:07:39,252: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-03 01:07:39,256: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-10-03 01:07:39,257: INFO: common: created directory at: artifacts]
[2023-10-03 01:07:39,258: INFO: common: created directory at: artifacts/data_transformation]
[2023-10-03 01:07:39,258: INFO: 537558814: Starting Data Transformation Process...]
[2023-10-03 01:07:39,290: INFO: 537558814: Handling Null Values...]
[2023-10-03 01:07:39,293: INFO: 537558814: Dropping Columns: sl_no, ssc_b, hsc_b]
[2023-10-03 01:07:39,300: INFO: 537558814: Handling Outliers in Column: hsc_p]
[2023-10-03 01:07:39,308: INFO: 537558814: Encoding Labels for Columns: gender, workex, specialisation, status]
[2023-10-03 01:07:39,313: INFO: 537558814: Dummy Encoding Columns: hsc_s, degree_t]
[2023-10-03 01:07:39,325: INFO: 537558814: Splitting Data with Tes

In [47]:
x = pd.read_csv('/Users/harkiratchahal/Desktop/Coding/Tutorials/mlops_2/artifacts/data_transformation/train.csv')
x.head()

,gender,ssc_p,hsc_p,degree_p,workex,etest_p,specialisation,mba_p,status,salary,dummy_hsc_s_Commerce,dummy_hsc_s_Science,dummy_degree_t_Others,dummy_degree_t_Sci&Tech
0,0,47.00,55.00,65.0,0,62.0,1,65.04,0,0.0,0,1,0,0
1,0,40.89,45.83,53.0,0,71.2,1,65.49,0,0.0,1,0,0,0
2,1,74.00,62.00,68.0,0,74.0,0,57.99,1,268000.0,0,1,0,0
3,1,52.00,65.00,57.0,1,75.0,0,59.81,0,0.0,0,0,1,0
4,1,68.00,76.00,74.0,0,80.0,0,63.59,1,360000.0,1,0,0,0


In [48]:
x.isna().sum()

gender                     0
ssc_p                      0
hsc_p                      0
degree_p                   0
workex                     0
etest_p                    0
specialisation             0
mba_p                      0
status                     0
salary                     0
dummy_hsc_s_Commerce       0
dummy_hsc_s_Science        0
dummy_degree_t_Others      0
dummy_degree_t_Sci&Tech    0
dtype: int64